### Preprocessing of Cushing, Oklahoma wastewater disposal and seismicity data



In [1]:
# Load relevant packages.
import numpy as np
import h5py
import pandas as pd

### Seismic data 

In [2]:
# Seismic data.
with h5py.File('../datasets_Jul_7/cushing_oklahoma/cushingSeismic.hdf5','r') as hdf:
    ls = list(hdf.keys())
    print("List of columns in this file: \n", ls)
    epoch = np.array(hdf.get('epoch'))
    latitude = np.array(hdf.get('latitude'))
    longitude = np.array(hdf.get('longitude'))    
    depth = np.array(hdf.get('depth'))
    easting = np.array(hdf.get('easting'))
    northing = np.array(hdf.get('northing'))
    magnitude = np.array(hdf.get('magnitude'))

List of columns in this file: 
 ['b_value', 'b_value_epoch', 'depth', 'dpdt', 'easting', 'epoch', 'eventID', 'latitude', 'longitude', 'magnitude', 'northing']


In [3]:
# Set up and write seismic catalog.
seismic = pd.DataFrame({'epoch': epoch,
                        'latitude': latitude,
                        'longitude': longitude,
                        'depth': depth,
                        'easting': easting,
                        'northing': northing,
                        'magnitude': magnitude
                      })

In [4]:
seismic.sample(8)

,epoch,latitude,longitude,depth,easting,northing,magnitude
891,1.459508e+09,35.93570,-96.75237,3.927,702754.187443,3.979151e+06,1.84
826,1.457266e+09,35.92937,-96.73756,4.615,704106.666266,3.978480e+06,1.81
207,1.412900e+09,35.95173,-96.78008,6.458,700213.663590,3.980872e+06,1.64
210,1.412909e+09,35.94608,-96.78452,3.979,699827.395129,3.980237e+06,1.30
157,1.412702e+09,35.94511,-96.75716,5.000,702298.023893,3.980185e+06,1.42
473,1.425364e+09,35.94862,-96.78442,3.892,699830.016245,3.980519e+06,1.90
911,1.459747e+09,35.93637,-96.75243,4.482,702747.061990,3.979226e+06,2.57
752,1.441245e+09,35.94864,-96.77019,4.832,701113.615629,3.980550e+06,1.42


In [5]:
seismic.to_csv('../staging/temporal/cushing_oklahoma/seismic.csv', index=None)

### Load well positions, injection data, and pressurization rate data.

In [6]:
# Epoch (s)
t = np.array(pd.read_csv('../datasets_Jul_7/cushing_oklahoma/pressure_table/t.csv', header=None)[0])
print('No. of samples along t = %d'% (len(t)))

# Well easting (m)
x = np.array(pd.read_csv('../datasets_Jul_7/cushing_oklahoma/pressure_table/x.csv', header=None)[0])
print('No. of samples along x = %d'% (len(x)))

# Well northing (m)
y = np.array(pd.read_csv('../datasets_Jul_7/cushing_oklahoma/pressure_table/y.csv', header=None)[0])
print('No. of samples along y = %d'% (len(y)))

# Well depth (m)
z = np.array(pd.read_csv('../datasets_Jul_7/cushing_oklahoma/pressure_table/z.csv', header=None)[0])
print('No. of samples along z = %d'% (len(z)))

No. of samples along t = 25000
No. of samples along x = 1
No. of samples along y = 1
No. of samples along z = 1


It looks like we have only 1 injection well.

In [7]:
# Pressurization rate data.
dpdt = np.array(pd.read_csv('../datasets_Jul_7/cushing_oklahoma/pressure_table/dpdt.csv', header=None)[0])
print('No. of dp/dt samples = %d'% (len(dpdt)))

No. of dp/dt samples = 25000


In [8]:
# Flow rate data
df_flow = pd.read_csv('../datasets_Jul_7/cushing_oklahoma/cushingClusterCenterDPDT.csv')

In [9]:
df_flow.head()

,epoch,flow_rate
0,7.862378e+08,0.0
1,7.862669e+08,0.0
2,7.862959e+08,0.0
3,7.863249e+08,0.0
4,7.863539e+08,0.0


In [10]:
injection = np.array(df_flow['flow_rate'])

### TO DO: Compute pressures using radial flow model in orion_light.

In [11]:
# Radial flow parameters

viscosity = 1.0       # cD
permeability = 194.5  # mD
storativity = 0.0014 
reservoir_depth = 100 # m

# min_radius = ?

A simple workaround is to compute pressures by integrating dp/dt over t.

In [12]:
from orion_light.pressure_table import PressureTableModel

In [13]:
model = PressureTableModel()

In [14]:
model.load_array(dpdt=dpdt, x=x, y=y, z=z, t=t)

In [15]:
model.table_data

{'dpdt': array([[[[0.        , 0.        , 0.        , ..., 0.01754773,
           0.017566  , 0.01758427]]]]),
 'x': array([0.]),
 'y': array([0.]),
 'z': array([0.]),
 't': array([7.86237840e+08, 7.86266854e+08, 7.86295869e+08, ...,
        1.51202621e+09, 1.51205523e+09, 1.51208424e+09]),
 'pressure': array([[[[      0.        ,       0.        ,       0.        , ...,
           7153543.25126303, 7154052.6511425 , 7154562.58097576]]]])}

Injection pressure is now computed.

### Set up and write injection data catalog.

In [16]:
# Set up injection catalog.
injection_cat = pd.DataFrame({'epoch': t,
                              'injection': injection,
                              'pressure': model.table_data['pressure'].squeeze(),
                              'dpdt': dpdt
                            })

In [17]:
injection_cat.tail(10)

,epoch,injection,pressure,dpdt
24990,1.511823e+09,0.017420,7.149992e+06,0.017420
24991,1.511852e+09,0.017438,7.150498e+06,0.017438
24992,1.511881e+09,0.017456,7.151004e+06,0.017456
24993,1.511910e+09,0.017475,7.151511e+06,0.017475
24994,1.511939e+09,0.017493,7.152018e+06,0.017493
24995,1.511968e+09,0.017511,7.152526e+06,0.017511
24996,1.511997e+09,0.017529,7.153034e+06,0.017529
24997,1.512026e+09,0.017548,7.153543e+06,0.017548
24998,1.512055e+09,0.017566,7.154053e+06,0.017566
24999,1.512084e+09,0.017584,7.154563e+06,0.017584


In [18]:
injection_cat.to_csv('../staging/temporal/cushing_oklahoma/injection.csv', index=None)